 Note: This notebook is shared in a clean format (without cell outputs) for readability and version control.  
 The model was trained and evaluated separately, and the results are summarized in the README file.

## Weather Image Classification using Convolutional Neural Networks (CNN)

This project aims to classify weather conditions from images by building a Convolutional Neural Network (CNN) model.  
For this purpose, the Kaggle **5-Class Weather Status Image Classification** dataset is utilized for training and evaluation.

## Importing Required Libraries

In [ ]:
!pip install opencv-python

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


## Dataset Preparation and Splitting

The dataset is preprocessed and divided into training, validation, and test subsets to improve model generalization and enable reliable performance evaluation.

In [ ]:
import os
import shutil
import random


original_dataset_dir = '/kaggle/input/5class-weather-status-image-classification/data'  

# Yeni dataset klasörü (train/validation/test olacak)
base_dir = '/kaggle/working/dataset_split'  
os.makedirs(base_dir, exist_ok=True)

# Train/Validation/Test klasörlerini oluştur
for split in ['train', 'validation', 'test']:
    split_dir = os.path.join(base_dir, split)
    os.makedirs(split_dir, exist_ok=True)

    for class_name in os.listdir(original_dataset_dir):
        class_split_dir = os.path.join(split_dir, class_name)
        os.makedirs(class_split_dir, exist_ok=True)

        class_dir = os.path.join(original_dataset_dir, class_name)
        images = os.listdir(class_dir)
        random.shuffle(images)

        # %70 train, %20 validation, %10 test
        train_split = int(0.7 * len(images))
        val_split = int(0.9 * len(images))

        train_images = images[:train_split]
        val_images = images[train_split:val_split]
        test_images = images[val_split:]

        # Dosyaları kopyala
        for img in train_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(split_dir, class_name, img))
        for img in val_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(split_dir, class_name, img))
        for img in test_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(split_dir, class_name, img))

print("Dataset train/validation/test olarak ayrıldı!")


## Data Preprocessing and Visualization

The dataset is normalized to the [0, 1] range.  
Class labels are defined, and sample images are visualized to better understand the data distribution.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os 
import numpy as np 

# 1. ImageDataGenerator Tanımlamaları 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,           
    width_shift_range=0.25,      
    height_shift_range=0.25,     
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

#  2. Data Generator Akışları
train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=(64, 64),
    batch_size=128,
    class_mode='categorical',
    shuffle=True
)

validation_generator = val_test_datagen.flow_from_directory(
    os.path.join(base_dir, 'validation'),
    target_size=(64, 64),
    batch_size=128,
    class_mode='categorical',
    shuffle=False 
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=(64, 64),
    batch_size=128,
    class_mode='categorical',
    shuffle=False
)

print(f"\nEğitim verisi: {train_generator.n} resim")
print(f"Doğrulama verisi: {validation_generator.n} resim")
print(f"Test verisi: {test_generator.n} resim")

## Building the Neural Network (CNN Architecture)

The model is constructed using `tf.keras.Sequential()`.  
The architecture consists of the following layers:

- Conv2D (Convolutional Layers) for feature extraction  
- MaxPooling2D (Pooling Layers) for spatial downsampling  
- Flatten layer for vectorization  
- Dense (Fully Connected Layers) for classification  
- Dropout layer for regularization and overfitting prevention

In [ ]:
from tensorflow.keras import Model, Input, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau # Dinamik hız kontrolü için eklendi

inputs = Input(shape=(64, 64, 3))

# BLOK 1
x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(inputs) 
x = layers.MaxPooling2D(2,2)(x)


# BLOK 2
x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x) 
x = layers.MaxPooling2D(2,2)(x)
x = layers.Dropout(0.25)(x) 

# BLOK 3
x = layers.Conv2D(256, (3,3), activation='relu', padding='same')(x) 
x = layers.MaxPooling2D(2,2)(x)
x = layers.Dropout(0.25)(x)

# BLOK 4
x = layers.Conv2D(512, (3,3), activation='relu', padding='same')(x) 
x = layers.MaxPooling2D(2,2)(x)

x = layers.Flatten()(x)

# TAM BAĞLANTILI KATMANLAR
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.4)(x) 

outputs = layers.Dense(5, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Öğrenme hızı 0.0005 olarak ayarlandı
optimizer = Adam(learning_rate=0.0005) 

model.compile(
    optimizer=optimizer, 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)


## Model Training

The model is trained using the `model.fit()` method.  
After the training process, the trained model is saved using `model.save()` for future use and reproducibility.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Callbacks tanımlamaları
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10) 

# Eğitimi başlat
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=100, # Veya daha fazla
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[early_stopping, reduce_lr] 
)
model.save('final_best_model.h5')

## Visualization of Training Results and Accuracy

Training and validation results are visualized using graphical plots to analyze the learning process.  
Model accuracy is calculated to evaluate the overall performance during training.

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Eğitim Doğruluğu')
plt.plot(epochs_range, val_acc, label='Doğrulama Doğruluğu')
plt.legend(loc='lower right')
plt.title('Eğitim ve Doğrulama Doğruluğu')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Eğitim Kaybı')
plt.plot(epochs_range, val_loss, label='Doğrulama Kaybı')
plt.legend(loc='upper right')
plt.title('Eğitim ve Doğrulama Kaybı')

plt.show()


## Performance Evaluation

The model’s performance is evaluated on the test dataset using appropriate evaluation metrics to assess its generalization capability.

In [ ]:
test_loss, test_acc = model.evaluate(
    test_generator,
    steps=len(test_generator),
    verbose=2
)

print(f"Test kaybı: {test_loss:.4f}")
print(f"Test doğruluk: {test_acc*100:.2f}%")


## Prediction and Inference

Predictions are generated using the `model.predict()` method on unseen sample images to test the model’s classification capability.

In [ ]:

sample_batch, _ = next(test_generator)
sample_image = sample_batch[0:1]  

# Tahmin yap
pred = model.predict(sample_image)  # output: [[0.1, 0.2, 0.05, 0.6, 0.05]]

# Tahmin edilen sınıf
predicted_class_index = np.argmax(pred, axis=1)[0]

# Tahmin olasılığı
predicted_prob = np.max(pred)

# Sınıf isimleri
class_names = ['cloudy', 'foggy', 'rainy', 'snowy', 'sunny']
predicted_class_name = class_names[predicted_class_index]

print(f"Modelin tahmini sınıfı: {predicted_class_name}")
print(f"Tahmin olasılığı: {predicted_prob*100:.2f}%")


## Model Interpretability with Grad-CAM

Grad-CAM (Gradient-weighted Class Activation Mapping) is used to visualize the regions of the input images that the model focuses on during classification.  
This technique helps interpret the model’s decision-making process by highlighting the most influential areas in the image.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 

try:
    # Model, eğitimden sonra kaydedilen dosyadan yükleniyor.
    model = tf.keras.models.load_model('final_best_model.h5')
    # Son katman adını model yüklendikten sonra alıyoruz
    LAST_CONV_LAYER_NAME = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            LAST_CONV_LAYER_NAME = layer.name
            break
    print("✅ Model, kaydedilen ağırlıklarla başarıyla yüklendi.")
    print("Grad-CAM için kullanılacak katman:", LAST_CONV_LAYER_NAME)

except Exception as e:
    # Yükleme başarısız olursa, Grad-CAM çalışmaz.
    print("❌ Kayıtlı model yüklenemedi. Lütfen önce 'Run All' ile modeli eğitin ve kaydedin.")
    print(f"Hata: {e}")
    # Eğer yüklenemezse, kodun geri kalanının çalışmasını durdururuz
    raise SystemExit(0) 

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    img_array = tf.convert_to_tensor(img_array, dtype=tf.float32)
    grad_model = Model(
        [model.input], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Heatmap Görselleştirme 

def display_gradcam(img_array, heatmap, title="Grad-CAM"):
    img = np.uint8(255 * img_array) if img_array.max() <= 1 else np.uint8(img_array)
    heatmap = cv2.resize(heatmap, (int(img.shape[1]), int(img.shape[0])), interpolation=cv2.INTER_LINEAR)
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)
    
    # Görseli göster
    plt.figure(figsize=(6, 4))
    plt.imshow(superimposed_img)
    plt.title(title)
    plt.axis('off')
    plt.show()


CLASS_NAMES = ['cloudy', 'foggy', 'rainy', 'snowy', 'sunny']


# Grad-CAM Uygulaması (Rastgele 15 Tahmin)

print("\n--- Grad-CAM ile Model Yorumlanabilirliği (Rastgele Tahminler) ---")

iterator = iter(test_generator) 


for batch_num in range(5): 
    try:
        images, labels = next(iterator)
    except StopIteration:
        break

    print(f"\n--- Batch {batch_num + 1} İşleniyor ---")
    
    for i in range(min(len(images), 3)): 
        img_array = images[i]
        true_index = tf.argmax(labels[i]).numpy()
        true_label = CLASS_NAMES[true_index]
        
        img_for_pred = tf.expand_dims(img_array, axis=0)
        img_for_pred = tf.convert_to_tensor(img_for_pred, dtype=tf.float32)
        
        # Tahmin 
        predictions = model.predict(img_for_pred, verbose=0)
        predicted_index = np.argmax(predictions[0])
        predicted_label = CLASS_NAMES[predicted_index]

        # Grad-CAM'i çalıştır 
        heatmap = make_gradcam_heatmap(
            img_for_pred, model, LAST_CONV_LAYER_NAME, pred_index=predicted_index
        )
        
        
        if predicted_index == true_index:
            result_status = "✅ DOĞRU"
        else:
            result_status = "❌ YANLIŞ"

        title = f"{result_status} | Tahmin: {predicted_label.upper()} | Gerçek: {true_label.upper()}"
        
        print("-" * 30)
        print(f"Görüntü {i+1} | Gerçek: {true_label}, Tahmin: {predicted_label} ({result_status})")
        display_gradcam(img_array, heatmap, title)